In [14]:
import torch

BATCH_SIZE = 4 # increase / decrease according to GPU memeory
RESIZE_TO = 512 # resize the image for training and transforms
NUM_EPOCHS = 10 # number of epochs to train for

DEVICE = torch.device('cpu')

# training images and XML files directory
TRAIN_DIR = '../data/regression/training'
# validation images and XML files directory
VALID_DIR = '../data/regression/validation'

# whether to visualize images after crearing the data loaders
VISUALIZE_TRANSFORMED_IMAGES = False

# location to save model and plots
OUT_DIR = '../results/regression'
SAVE_PLOTS_EPOCH = 2 # save loss plots after these many epochs
SAVE_MODEL_EPOCH = 2 # save model after these many epochs

In [15]:
import albumentations as A
import cv2
import numpy as np

from albumentations.pytorch import ToTensorV2

# this class keeps track of the training and validation loss values...
# ... and helps to get the average for each epoch as well
class Averager:
    def __init__(self):
        self.current_total = 0.0
        self.iterations = 0.0
        
    def send(self, value):
        self.current_total += value
        self.iterations += 1
    
    @property
    def value(self):
        if self.iterations == 0:
            return 0
        else:
            return 1.0 * self.current_total / self.iterations
    
    def reset(self):
        self.current_total = 0.0
        self.iterations = 0.0

In [16]:
def collate_fn(batch):
    """
    To handle the data loading as different images may have different number 
    of objects and to handle varying size tensors as well.
    """
    return tuple(zip(*batch))

In [17]:
# define the training tranforms
def get_train_transform():
    return A.Compose([
        A.Flip(0.5),
        A.RandomRotate90(0.5),
        A.MotionBlur(p=0.2),
        A.MedianBlur(blur_limit=3, p=0.1),
        A.Blur(blur_limit=3, p=0.1),
        ToTensorV2(p=1.0),
    ])

# define the validation transforms
def get_valid_transform():
    return A.Compose([
        ToTensorV2(p=1.0),
    ], bbox_params={
        'format': 'pascal_voc', 
        'label_fields': ['labels']
    })

# define the negative sample transforms
def get_negative_sample_transform():
    return A.Compose([
        ToTensorV2(p=1.0),
    ])

In [18]:
def show_tranformed_image(train_loader):
    """
    This function shows the transformed images from the `train_loader`.
    Helps to check whether the tranformed images along with the corresponding
    labels are correct or not.
    Only runs if `VISUALIZE_TRANSFORMED_IMAGES = True` in config.py.
    """
    if len(train_loader) > 0:
        for i in range(1):
            images, targets = next(iter(train_loader))
            images = list(image.to(DEVICE) for image in images)
            targets = [{k: v.to(DEVICE) for k, v in t.items()} for t in targets]
            boxes = targets[i]['boxes'].cpu().numpy().astype(np.int32)
            sample = images[i].permute(1, 2, 0).cpu().numpy()
            for box in boxes:
                cv2.rectangle(sample,
                            (box[0], box[1]),
                            (box[2], box[3]),
                            (0, 0, 255), 2)
            cv2.imshow('Transformed image', sample)
            cv2.waitKey(0)
            cv2.destroyAllWindows()

In [19]:
import torch
import cv2
import numpy as np
import os
import glob as glob
import pandas as pd

from torch.utils.data import Dataset, DataLoader

In [103]:
# the dataset class
class WormDataset(Dataset):
    def __init__(self, dir_path, width, height, transforms, negative_sample_transforms):
        self.transforms = transforms
        self.negative_sample_transforms = negative_sample_transforms
        self.dir_path = dir_path
        self.height = height
        self.width = width
        
        # get all the image paths in sorted order
        self.image_paths = glob.glob(f"{self.dir_path}/*[jpg|jpeg]")
        self.all_images = [image_path.split('/')[-1] for image_path in self.image_paths]
        self.all_images = sorted(self.all_images)

    def __getitem__(self, idx):
        # capture the image name and the full image path
        image_name = self.all_images[idx]
        image_path = os.path.join(self.dir_path, image_name)

        # read the image
        image = cv2.imread(image_path)
        # convert BGR to RGB color format
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB).astype(np.float32)
        image_resized = cv2.resize(image, (self.width, self.height))
        image_resized /= 255.0
        
        # capture the corresponding XML file for getting the annotations
        annotations_file = image_name.split('.')[0] + '.txt'
        annotations_file_path = os.path.join(self.dir_path, annotations_file)
        
        # box coordinates for xml files are extracted and corrected for image size given

        annotations_df = pd.read_csv(annotations_file_path, sep='\t', names=["type", 'total'])

        # prepare the final `target` dictionary
        is_negative_sample = len(annotations_df) == 0
        target_abw = annotations_df[annotations_df['type'] == 'abw']['total'].iloc[0] if len(annotations_df[annotations_df['type'] == 'abw']) != 0 else 0
        target_pbw = annotations_df[annotations_df['type'] == 'pbw']['total'].iloc[0] if len(annotations_df[annotations_df['type'] == 'pbw']) != 0 else 0

        # apply the image transforms
        if not is_negative_sample:
            sample = self.transforms(image = image_resized)
            image_resized = sample['image']
        elif is_negative_sample:
            sample = self.negative_sample_transforms(image=image_resized)
            image_resized = sample['image']
        
        return image_resized, target_abw, target_pbw

    def __len__(self):
        return len(self.all_images)

In [104]:
# prepare the final datasets and data loaders
train_dataset = WormDataset(TRAIN_DIR, RESIZE_TO, RESIZE_TO, get_train_transform(), get_negative_sample_transform())
valid_dataset = WormDataset(VALID_DIR, RESIZE_TO, RESIZE_TO, get_valid_transform(), get_negative_sample_transform())
train_loader = DataLoader(
    train_dataset,
    batch_size=BATCH_SIZE,
    shuffle=True,
    num_workers=0,
    collate_fn=collate_fn
)
valid_loader = DataLoader(
    valid_dataset,
    batch_size=BATCH_SIZE,
    shuffle=False,
    num_workers=0,
    collate_fn=collate_fn
)
print(f"Number of training samples: {len(train_dataset)}")
print(f"Number of validation samples: {len(valid_dataset)}\n")

Number of training samples: 8763
Number of validation samples: 974



In [22]:
# # sanity check of the Dataset pipeline with sample visualization
# dataset = WormDataset(
#     TRAIN_DIR, RESIZE_TO, RESIZE_TO, CLASSES
# )
# print(f"Number of training images: {len(dataset)}")

# # function to visualize a single sample
# def visualize_sample(image, target):
#     box = target['boxes'][0] if len(target['boxes']) > 0 else []
#     label = CLASSES[target['labels'][0]] if len(target['labels']) > 0 else []
#     if len(box) != 0:
#         cv2.rectangle(
#             image, 
#             (int(box[0]), int(box[1])), (int(box[2]), int(box[3])),
#             (0, 255, 0), 1
#         )
#         cv2.putText(
#             image, label, (int(box[0]), int(box[1]-5)), 
#             cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2
#         )
#     cv2.imshow('Image', image)
#     cv2.waitKey(0)
    
# NUM_SAMPLES_TO_VISUALIZE = 5
# for i in range(NUM_SAMPLES_TO_VISUALIZE):
#     image, target = dataset[i]
#     visualize_sample(image, target)

In [23]:
from tqdm.auto import tqdm

import torch
import matplotlib.pyplot as plt
import time

plt.style.use('ggplot')

In [113]:
# function for running training iterations
def train(train_data_loader, model, optimizer, criterion1, criterion2):
    print('Training')
    
    for i, data in enumerate(train_data_loader):
        optimizer.zero_grad()
        images, target_abw, target_pbw = data
        
        images = torch.stack(images)
        target_abw = torch.tensor(target_abw, dtype=torch.float32)
        target_pbw = torch.tensor(target_pbw, dtype=torch.float32)

        predicted_abw, predicted_pbw = model(images)

        loss1 = criterion1(predicted_abw, target_abw)
        loss2 = criterion2(predicted_pbw, target_pbw)
        loss = loss1 + loss2

        loss.backward()
        optimizer.step()
    
        # update the loss value beside the progress bar for each iteration
        print(f"Loss: {loss:.4f} ,Index: {i}, Total: {len(train_data_loader)}")
    return loss

In [114]:
# function for running validation iterations
def validate(valid_data_loader, model, criterion1, criterion2):
    print('Validating')
    
    for i, data in enumerate(valid_data_loader):
        images, target_abw, target_pbw = data

        images = torch.stack(images)
        target_abw = torch.tensor(target_abw)
        target_pbw = torch.tensor(target_pbw)

        with torch.no_grad():
            predicted_abw, predicted_pbw = model(images)

            loss1 = criterion1(predicted_abw, target_abw)
            loss2 = criterion2(predicted_pbw, target_pbw)
            loss = loss1 + loss2

        # update the loss value beside the progress bar for each iteration
        print(f"Loss: {loss:.4f} ,Index: {i}, Total: {len(valid_data_loader)}")
    return loss

In [117]:
import torch.nn as nn
from torchvision import models

class ImageRegressionNN(nn.Module):
    def __init__(self):
        super().__init__()
        layers = list(models.resnet34(weights="ResNet34_Weights.DEFAULT").children())[:-2]

        layers.append(nn.AdaptiveAvgPool2d(output_size=(1,2)))
        layers.append(nn.Flatten())
        layers.append(nn.BatchNorm1d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True))
        layers.append(nn.Dropout(p=0.50))
        layers.append(nn.Linear(1024, 512, bias=True))
        layers.append(nn.ReLU(inplace=True))
        layers.append(nn.BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True))
        layers.append(nn.Dropout(p=0.50))
        layers.append(nn.Linear(512, 16, bias=True))
        layers.append(nn.ReLU(inplace=True))
        layers.append(nn.Linear(16,1))

        self.abw_model = nn.Sequential(*layers)
        self.pbw_model = nn.Sequential(*layers)

    def forward(self, x):
        number_of_abw = self.abw_model(x).squeeze(-1)
        number_of_pbw = self.pbw_model(x).squeeze(-1)

        return torch.round(number_of_abw), torch.round(number_of_pbw)

In [118]:
# initialize the model and move to the computation device
model = ImageRegressionNN()
model = model.to(DEVICE)
# get the model parameters
params = [p for p in model.parameters() if p.requires_grad]
# define the optimizer
optimizer = torch.optim.Adam(params, lr=0.001, betas=(0.9,0.99), eps=1e-5)
criterion1 = torch.nn.MSELoss() 
criterion2 = torch.nn.MSELoss() 

# initialize the Averager class
train_loss_hist = Averager()
val_loss_hist = Averager()
# train and validation loss lists to store loss values of all...
# ... iterations till ena and plot graphs for all iterations
train_loss_list = []
val_loss_list = []

all_training_losses = []
all_validation_losses = []

# name to save the trained model with
MODEL_NAME = 'regression'

# whether to show transformed images from data loader or not
if VISUALIZE_TRANSFORMED_IMAGES:
    show_tranformed_image(train_loader)

# start the training epochs
for epoch in range(NUM_EPOCHS):
    print(f"\nEPOCH {epoch+1} of {NUM_EPOCHS}")

    # reset the training and validation loss histories for the current epoch
    train_loss_hist.reset()
    val_loss_hist.reset()

    # start timer and carry out training and validation
    start = time.time()
    train_loss = train(train_loader, model, optimizer, criterion1, criterion2)
    val_loss = validate(valid_loader, model, criterion1, criterion2)
    print(f"Epoch #{epoch} train loss: {train_loss_hist.value:.3f}")   
    print(f"Epoch #{epoch} validation loss: {val_loss_hist.value:.3f}")   
    end = time.time()
    print(f"Took {((end - start) / 60):.3f} minutes for epoch {epoch}")

    if (epoch+1) % SAVE_MODEL_EPOCH == 0: # save model after every n epochs
        torch.save(model.state_dict(), f"{OUT_DIR}/model{epoch+1}.pth")
        print('SAVING MODEL COMPLETE...\n')
    
    if (epoch+1) == NUM_EPOCHS: # save loss plots and model once at the end
        torch.save(model.state_dict(), f"{OUT_DIR}/model{epoch+1}.pth")
    
    all_training_losses.append(train_loss.value)
    all_validation_losses.append(val_loss.value)
    
    plt.close('all')


EPOCH 1 of 10
Training
Loss: 70.5000 ,Index: 0, Total: 2191
Loss: 22.2500 ,Index: 1, Total: 2191
Loss: 1107.5000 ,Index: 2, Total: 2191
Loss: 283.5000 ,Index: 3, Total: 2191
Loss: 6.5000 ,Index: 4, Total: 2191
Loss: 12.7500 ,Index: 5, Total: 2191
Loss: 1556.5000 ,Index: 6, Total: 2191
Loss: 3801.7500 ,Index: 7, Total: 2191
Loss: 2393.7500 ,Index: 8, Total: 2191
Loss: 27560.2500 ,Index: 9, Total: 2191
Loss: 2605.2500 ,Index: 10, Total: 2191
Loss: 0.7500 ,Index: 11, Total: 2191
Loss: 11261.2500 ,Index: 12, Total: 2191
Loss: 2874.5000 ,Index: 13, Total: 2191
Loss: 887.7500 ,Index: 14, Total: 2191
Loss: 34254.0000 ,Index: 15, Total: 2191
Loss: 1183.5000 ,Index: 16, Total: 2191
Loss: 1023.2500 ,Index: 17, Total: 2191
Loss: 18.0000 ,Index: 18, Total: 2191
Loss: 516.7500 ,Index: 19, Total: 2191
Loss: 2924.2500 ,Index: 20, Total: 2191
Loss: 29.0000 ,Index: 21, Total: 2191
Loss: 4848.5000 ,Index: 22, Total: 2191
Loss: 12.2500 ,Index: 23, Total: 2191
Loss: 114.2500 ,Index: 24, Total: 2191
Loss:

Invalid SOS parameters for sequential JPEG


Loss: 2227.2500 ,Index: 288, Total: 2191
Loss: 4.2500 ,Index: 289, Total: 2191
Loss: 250.7500 ,Index: 290, Total: 2191
Loss: 10090.5000 ,Index: 291, Total: 2191
Loss: 851.2500 ,Index: 292, Total: 2191
Loss: 32.5000 ,Index: 293, Total: 2191
Loss: 4242.0000 ,Index: 294, Total: 2191
Loss: 76.7500 ,Index: 295, Total: 2191
Loss: 217.0000 ,Index: 296, Total: 2191
Loss: 9.2500 ,Index: 297, Total: 2191
Loss: 47.5000 ,Index: 298, Total: 2191
Loss: 1348.5000 ,Index: 299, Total: 2191
Loss: 260.7500 ,Index: 300, Total: 2191
Loss: 7.5000 ,Index: 301, Total: 2191
Loss: 29.5000 ,Index: 302, Total: 2191
Loss: 528.5000 ,Index: 303, Total: 2191
Loss: 85.0000 ,Index: 304, Total: 2191
Loss: 36.5000 ,Index: 305, Total: 2191
Loss: 13.2500 ,Index: 306, Total: 2191
Loss: 1.0000 ,Index: 307, Total: 2191
Loss: 5.5000 ,Index: 308, Total: 2191
Loss: 5333.5000 ,Index: 309, Total: 2191
Loss: 1057.0000 ,Index: 310, Total: 2191
Loss: 0.5000 ,Index: 311, Total: 2191
Loss: 1315.5000 ,Index: 312, Total: 2191
Loss: 805.7

Invalid SOS parameters for sequential JPEG


Loss: 315.2500 ,Index: 506, Total: 2191
Loss: 171.5000 ,Index: 507, Total: 2191
Loss: 100.5000 ,Index: 508, Total: 2191
Loss: 116.7500 ,Index: 509, Total: 2191
Loss: 80.2500 ,Index: 510, Total: 2191
Loss: 20497.5000 ,Index: 511, Total: 2191
Loss: 29.2500 ,Index: 512, Total: 2191
Loss: 352.7500 ,Index: 513, Total: 2191
Loss: 779.7500 ,Index: 514, Total: 2191
Loss: 11078.7500 ,Index: 515, Total: 2191
Loss: 34.2500 ,Index: 516, Total: 2191
Loss: 13.0000 ,Index: 517, Total: 2191
Loss: 7130.5000 ,Index: 518, Total: 2191
Loss: 6.5000 ,Index: 519, Total: 2191
Loss: 2661.2500 ,Index: 520, Total: 2191
Loss: 16.2500 ,Index: 521, Total: 2191
Loss: 5.0000 ,Index: 522, Total: 2191
Loss: 90.2500 ,Index: 523, Total: 2191
Loss: 6.5000 ,Index: 524, Total: 2191
Loss: 40.7500 ,Index: 525, Total: 2191
Loss: 93.2500 ,Index: 526, Total: 2191
Loss: 383.7500 ,Index: 527, Total: 2191
Loss: 328.5000 ,Index: 528, Total: 2191
Loss: 14.7500 ,Index: 529, Total: 2191
Loss: 951.7500 ,Index: 530, Total: 2191
Loss: 11.

Invalid SOS parameters for sequential JPEG


Loss: 12.5000 ,Index: 553, Total: 2191
Loss: 34.2500 ,Index: 554, Total: 2191
Loss: 59.5000 ,Index: 555, Total: 2191
Loss: 2926.0000 ,Index: 556, Total: 2191
Loss: 2.5000 ,Index: 557, Total: 2191
Loss: 133.2500 ,Index: 558, Total: 2191
Loss: 9.0000 ,Index: 559, Total: 2191
Loss: 56.2500 ,Index: 560, Total: 2191
Loss: 15.5000 ,Index: 561, Total: 2191
Loss: 94.2500 ,Index: 562, Total: 2191
Loss: 304.7500 ,Index: 563, Total: 2191
Loss: 1631.5000 ,Index: 564, Total: 2191
Loss: 2.0000 ,Index: 565, Total: 2191
Loss: 18.2500 ,Index: 566, Total: 2191
Loss: 1.2500 ,Index: 567, Total: 2191
Loss: 27.2500 ,Index: 568, Total: 2191
Loss: 83.7500 ,Index: 569, Total: 2191
Loss: 12.2500 ,Index: 570, Total: 2191
Loss: 31.5000 ,Index: 571, Total: 2191
Loss: 13.2500 ,Index: 572, Total: 2191
Loss: 26.7500 ,Index: 573, Total: 2191
Loss: 2759.2500 ,Index: 574, Total: 2191
Loss: 150.2500 ,Index: 575, Total: 2191
Loss: 42.2500 ,Index: 576, Total: 2191
Loss: 15.5000 ,Index: 577, Total: 2191
Loss: 111.5000 ,Inde

Invalid SOS parameters for sequential JPEG


Loss: 132.2500 ,Index: 857, Total: 2191
Loss: 1974.5000 ,Index: 858, Total: 2191
Loss: 2.7500 ,Index: 859, Total: 2191
Loss: 564.7500 ,Index: 860, Total: 2191
Loss: 46.7500 ,Index: 861, Total: 2191
Loss: 402.5000 ,Index: 862, Total: 2191
Loss: 30.2500 ,Index: 863, Total: 2191
Loss: 210.5000 ,Index: 864, Total: 2191
Loss: 103.7500 ,Index: 865, Total: 2191
Loss: 57.2500 ,Index: 866, Total: 2191
Loss: 241.7500 ,Index: 867, Total: 2191
Loss: 24.2500 ,Index: 868, Total: 2191
Loss: 45.5000 ,Index: 869, Total: 2191
Loss: 183.2500 ,Index: 870, Total: 2191
Loss: 766.7500 ,Index: 871, Total: 2191
Loss: 240.5000 ,Index: 872, Total: 2191
Loss: 83.7500 ,Index: 873, Total: 2191
Loss: 41.2500 ,Index: 874, Total: 2191
Loss: 144.7500 ,Index: 875, Total: 2191
Loss: 120.5000 ,Index: 876, Total: 2191
Loss: 58.5000 ,Index: 877, Total: 2191
Loss: 71.5000 ,Index: 878, Total: 2191
Loss: 129.5000 ,Index: 879, Total: 2191
Loss: 2.2500 ,Index: 880, Total: 2191
Loss: 33.0000 ,Index: 881, Total: 2191
Loss: 12.7500

ValueError: Your 'label_fields' are not valid - them must have same names as params in dict

In [24]:
train_val_df = pd.DataFrame({
    'epochs': np.arange(1, 11, 1, dtype=int),
    'training_loss': all_training_losses,
    'validation_loss': all_validation_losses,
})
train_val_df.to_csv('../results/rcnn/train_val_results.csv', index=False)
train_val_df

,epochs,training_loss,validation_loss
0,1,0.462712,0.389970
1,2,0.396065,0.370712
2,3,0.380058,0.359849
3,4,0.372427,0.350515
4,5,0.363129,0.353558
5,6,0.359340,0.343766
6,7,0.356544,0.343392
7,8,0.352014,0.334505
8,9,0.347437,0.336476
9,10,0.346282,0.335564


In [16]:
import numpy as np
import cv2
import torch
import glob as glob

# set the computation device
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
# load the model and the trained weights
model = ImageRegressionNN().to(device)
model.load_state_dict(torch.load(
    f'{OUT_DIR}/model10.pth', map_location=device
))
model.eval()

FasterRCNN(
  (transform): GeneralizedRCNNTransform(
      Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
      Resize(min_size=(800,), max_size=1333, mode='bilinear')
  )
  (backbone): BackboneWithFPN(
    (body): IntermediateLayerGetter(
      (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (layer1): Sequential(
        (0): Bottleneck(
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
       

In [17]:
# directory where all the images are present
DIR_TEST = '../data/rcnn/testing/'
test_images = glob.glob(f"{DIR_TEST}/*")
print(f"Test instances: {len(test_images)}")

# define the detection threshold...
# ... any detection having score below this will be discarded
detection_threshold = 0.5

Test instances: 2803


In [18]:
results = []
for i in range(len(test_images)):
    # get the image file name for saving output later on
    image_name = test_images[i].split('/')[-1].split('.')[0]
    image = cv2.imread(test_images[i])
    orig_image = image.copy()
    # BGR to RGB
    image = cv2.cvtColor(orig_image, cv2.COLOR_BGR2RGB).astype(np.float32)
    # make the pixel range between 0 and 1
    image /= 255.0
    # bring color channels to front
    image = np.transpose(image, (2, 0, 1)).astype(np.float)
    # convert to tensor
    image = torch.tensor(image, dtype=torch.float)
    # add batch dimension
    image = torch.unsqueeze(image, 0)
    with torch.no_grad():
        outputs = model(image)
    
    # load all detection to CPU for further operations
    outputs = [{k: v.to('cpu') for k, v in t.items()} for t in outputs]
    # carry further only if there are detected boxes
    if len(outputs[0]['boxes']) != 0:
        boxes = outputs[0]['boxes'].data.numpy()
        scores = outputs[0]['scores'].data.numpy()
        # filter out boxes according to `detection_threshold`
        boxes = boxes[scores >= detection_threshold].astype(np.int32)
        draw_boxes = boxes.copy()
        # get all the predicted class names
        pred_classes = [CLASSES[i] for i in outputs[0]['labels'].numpy()]
        
        # draw the bounding boxes and write the class name on top of it
        for j, box in enumerate(draw_boxes):
            cv2.rectangle(orig_image,
                        (int(box[0]), int(box[1])),
                        (int(box[2]), int(box[3])),
                        (0, 0, 255), 2)
            cv2.putText(orig_image, pred_classes[j], 
                        (int(box[0]), int(box[1]-5)),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 
                        2, lineType=cv2.LINE_AA)
            results.append({
                "image": test_images[i].split('/')[-1],
                "bounding_box": box,
                "prediction": pred_classes[j]
            })

        # cv2.imshow('Prediction', orig_image)
        # cv2.waitKey(1)
        cv2.imwrite(f"{OUT_DIR}/test_predictions/{image_name}.jpg", orig_image,)
    print(f"Image {i+1} done...")
    print('-'*50)

/var/folders/n2/bwdv1zhn55ngx2l75t_7jwkw0000gn/T/ipykernel_10737/1240347181.py:12: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  image = np.transpose(image, (2, 0, 1)).astype(np.float)


Image 1 done...
--------------------------------------------------
Image 2 done...
--------------------------------------------------
Image 3 done...
--------------------------------------------------
Image 4 done...
--------------------------------------------------
Image 5 done...
--------------------------------------------------
Image 6 done...
--------------------------------------------------
Image 7 done...
--------------------------------------------------
Image 8 done...
--------------------------------------------------
Image 9 done...
--------------------------------------------------
Image 10 done...
--------------------------------------------------
Image 11 done...
--------------------------------------------------
Image 12 done...
--------------------------------------------------
Image 13 done...
--------------------------------------------------
Image 14 done...
--------------------------------------------------
Image 15 done...
----------------------------------------

Invalid SOS parameters for sequential JPEG


Image 928 done...
--------------------------------------------------
Image 929 done...
--------------------------------------------------
Image 930 done...
--------------------------------------------------
Image 931 done...
--------------------------------------------------
Image 932 done...
--------------------------------------------------
Image 933 done...
--------------------------------------------------
Image 934 done...
--------------------------------------------------
Image 935 done...
--------------------------------------------------
Image 936 done...
--------------------------------------------------
Image 937 done...
--------------------------------------------------
Image 938 done...
--------------------------------------------------
Image 939 done...
--------------------------------------------------
Image 940 done...
--------------------------------------------------
Image 941 done...
--------------------------------------------------
Image 942 done...
----------------

In [19]:
import pandas as pd

rcnn_results_df = pd.DataFrame(results)
rcnn_results_df

,image,bounding_box,prediction
0,id_3b59b5876e0a54fd5fd38188.jpg,"[381, 553, 405, 580]",pbw
1,id_3b59b5876e0a54fd5fd38188.jpg,"[409, 205, 434, 236]",pbw
2,id_3b59b5876e0a54fd5fd38188.jpg,"[608, 468, 631, 501]",pbw
3,id_3b59b5876e0a54fd5fd38188.jpg,"[744, 744, 772, 771]",pbw
4,id_3b59b5876e0a54fd5fd38188.jpg,"[587, 330, 609, 360]",pbw
...,...,...,...
39648,id_cbb6c9e7d0c2981f3932a3e1.jpg,"[1485, 211, 1886, 848]",abw
39649,id_cbb6c9e7d0c2981f3932a3e1.jpg,"[451, 43, 843, 782]",abw
39650,id_cbb6c9e7d0c2981f3932a3e1.jpg,"[1296, 1625, 1806, 2438]",abw
39651,id_cbb6c9e7d0c2981f3932a3e1.jpg,"[2283, 2133, 2924, 2990]",abw


In [20]:
rcnn_predictions_df = rcnn_results_df.groupby(['image', 'prediction']).size().reset_index(name='target')
rcnn_predictions_df['image'] = rcnn_predictions_df['image'].apply(lambda x: os.path.splitext(x)[0])
rcnn_predictions_df['Image_ID'] = (rcnn_predictions_df['image'] + "_" + rcnn_predictions_df['prediction']).str.lower()
rcnn_predictions_df = rcnn_predictions_df.drop(columns=['image', 'prediction'])
rcnn_predictions_df

,target,Image_ID
0,4,id_00332970f80fa9a47a39516d_abw
1,22,id_0035981bc3ae42eb5b57a317_pbw
2,9,id_005102f664b820f778291dee_abw
3,5,id_0066456f5fb2cd858c69ab39_abw
4,2,id_00887bebda26184c36e18e00_pbw
...,...,...
2389,11,id_ff40ad3bb245a697d799eeef_abw
2390,1,id_ff6b60935a327d1c2779bc5e_pbw
2391,8,id_ffad8f3773a4222f8fe5ba1a_pbw
2392,100,id_ffb65e6de900c49d8f2ef95a_pbw


In [21]:
test_df = pd.read_csv('../data/test.csv')

abw_df = test_df.copy(deep=True)
abw_df['worm_type'] = 'abw'
pbw_df = test_df.copy(deep=True)
pbw_df['worm_type'] = 'pbw'

full_test_df = pd.concat([abw_df, pbw_df], ignore_index=True)
full_test_df['image_id_worm'] = full_test_df['image_id_worm'].apply(lambda x: os.path.splitext(x)[0])
full_test_df['image_id_worm'] = (full_test_df['image_id_worm'] + "_" + full_test_df['worm_type']).str.lower()
full_test_df = full_test_df.drop(columns=['worm_type'])
full_test_df = full_test_df.rename(columns={"image_id_worm": "Image_ID"})
full_test_df

,Image_ID
0,id_00332970f80fa9a47a39516d_abw
1,id_0035981bc3ae42eb5b57a317_abw
2,id_005102f664b820f778291dee_abw
3,id_0066456f5fb2cd858c69ab39_abw
4,id_007159c1fa015ba6f394deeb_abw
...,...
5601,id_ffad8f3773a4222f8fe5ba1a_pbw
5602,id_ffb65e6de900c49d8f2ef95a_pbw
5603,id_ffbcb27fa549278f47505515_pbw
5604,id_ffc0e41e10b0c964d4a02811_pbw


In [22]:
final_testing_df = rcnn_predictions_df.copy(deep=True)

final_testing_df = pd.merge(full_test_df, final_testing_df, on="Image_ID", how="left")

final_testing_df['target'] = final_testing_df['target'].fillna(0)
final_testing_df['target'] = pd.to_numeric(final_testing_df['target'], downcast='integer')
final_testing_df = final_testing_df.sort_values(by='Image_ID').reset_index(drop=True)

final_testing_df.to_csv('../results/rcnn/rcnn_test_results_2.csv', index=False)
final_testing_df

,Image_ID,target
0,id_00332970f80fa9a47a39516d_abw,4
1,id_00332970f80fa9a47a39516d_pbw,0
2,id_0035981bc3ae42eb5b57a317_abw,0
3,id_0035981bc3ae42eb5b57a317_pbw,22
4,id_005102f664b820f778291dee_abw,9
...,...,...
5601,id_ffbcb27fa549278f47505515_pbw,0
5602,id_ffc0e41e10b0c964d4a02811_abw,0
5603,id_ffc0e41e10b0c964d4a02811_pbw,0
5604,id_fff8c253115aacded09ad7ed_abw,0
